# 1) Crawling main table 제작

In [1]:
#import selenium
#from selenium import webdriver
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests
import re
import pandas as pd
import numpy as np
import pymysql
import sqlalchemy

In [2]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='1234',db='esg_db', charset='utf8mb4')
cursor = conn.cursor()


In [3]:
def url_collect(keyword, date_st, date_ed, page_num):
    #keyword = input('검색 키워드를 입력하세요 : ')
    #date_st = input('기사 작성 시작 날짜를 입력하세요 : ')
    #date_ed = input('기사 작성 끝 날짜를 입력하세요 : ')
    date__st = date_st[:4] + '.' + date_st[4:6] + '.' + date_st[6:]
    date__ed = date_ed[:4] + '.' + date_ed[4:6] + '.' + date_ed[6:]

    final_df =  pd.DataFrame({'title' : [], 
                             'url' : [], 
                             'press' : [], 
                             'date' : [] })

    for i in tqdm(range(page_num)):

        #driver = webdriver.Chrome(executable_path='chromedriver.exe')
        url_keyword = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=' + keyword
        url_keyword_date = url_keyword + '&sort=0&photo=0&field=0&pd=3&ds=' + date__st + '&de=' + date__ed + '&cluster_rank=38&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from' + date_st + 'to' + date_ed + ',a:all&start=' + str(i) + '1'
        page = requests.get(url_keyword_date)  #네이버 뉴스 + 검색어 + 기간 적용해 접속
        #print(url_keyword_date)
        html = BeautifulSoup(page.text, "html.parser")

        time_list = []
        title_list = []
        url_list = []
        press_list = []

        # 기사작성 시간
        time_tag = html.find_all('span', {'class' : 'info'})
        for time in time_tag:
            #print(time.text)
            if '면'  in time.text:
                time = 'none'
            elif 'CHIEF EXECUTIVE' in time.text:
                 time = 'none'
            else :
                time_list.append(time.text)


        # 기사 제목
        ar_tag = html.find_all('a', {'class' : 'news_tit'})
        for ar in ar_tag:
            title_list.append(ar.get('title'))

        # 기사 url
        articles = html.select("li.bx > div > div > div.news_info > div.info_group")
        for article in articles:
            k = article.find_all('a', {'class' : 'info'})
            #print(k)
            try :
                url = k[1].get('href')
            except :
                url = k[0].get('href')    
            url_list.append(url)

        # 기사 언론사    
        ar_press = html.find_all('a', {'class' : 'info press'})
        for arr in ar_press:
            if '언론사 선정' in arr.text:
                press = arr.text[:-6]
            else : press = arr.text
            press_list.append(press)

        #print(time_list)
        data = pd.DataFrame({'title' : title_list, 
                             'url' : url_list, 
                             'press' : press_list, 
                             'date' : time_list })
        #print(data)
        final_df = pd.concat([final_df, data]).reset_index(drop=True)
        #print(final_df)
    return final_df

## * naver 뉴스 url을 가지지 않는 기사 지우기 + 10개 맞추기

In [4]:
def naver_cut_10(final_df):
    final_df['url_check'] = final_df['url'].apply(lambda x: 1 if 'naver' in x  else 0)
    final_df = final_df.loc[final_df['url_check'] == 1,['title','url','press','date']].reset_index(drop=True)
    final_df = final_df.head(10)
    return final_df

## ** 근 2년의 url 수집 모두 수집 

In [5]:
date = pd.read_excel('date.xlsx')   #날짜 데이터셋로드
date = date.iloc[366:].reset_index(drop=True) #2021년 만!!

In [6]:
date

,date
0,2021-01-01
1,2021-01-02
2,2021-01-03
3,2021-01-04
4,2021-01-05
...,...
360,2021-12-27
361,2021-12-28
362,2021-12-29
363,2021-12-30


In [6]:
url_dataset =  pd.DataFrame({'title' : [], 
                                 'url' : [], 
                                 'press' : [], 
                                 'date' : [] })

error_date = []
for j in tqdm(range(len(date))):
    # 날짜 셋팅
    start_date = str(date.iloc[j,0])[:4] + str(date.iloc[j,0])[5:7] + str(date.iloc[j,0])[8:10]
    end_date = start_date
    print( '날짜는',end_date)
    try : 
        temp_url = url_collect('esg', start_date, end_date, 10)
        temp_url = naver_cut_10(temp_url)
        url_dataset = pd.concat([url_dataset, temp_url]).reset_index(drop=True)
    #print(temp_url)
    #print(len(temp_url))
    #print('---------------------------------')
    
    sql = """
            INSERT INTO esg_url_crawl(
            title, url, press, date) VALUES(%s,%s,%s,%s)
           """
    cursor.executemany(sql, data)
    conn.commit()    
    
    except :
        print("***********************************"*2)
        print(end_date, '오류 있어 데이터 수집되지 않음')
        error_date.append(start_date)
        print("***********************************"*2)

print('총', len(error_date),'개의 날짜, 데이터 수집 되지 않음')

  0%|          | 0/365 [00:00<?, ?it/s]

날짜는 20210101


  0%|          | 0/10 [00:00<?, ?it/s]

날짜는 20210102


  0%|          | 0/10 [00:00<?, ?it/s]

...

날짜는 20211230


  0%|          | 0/10 [00:00<?, ?it/s]

날짜는 20211231


  0%|          | 0/10 [00:00<?, ?it/s]

총 8 개의 날짜, 데이터 수집 되지 않음


In [9]:
# 2021년 중 8개의 날짜는 제대로 수집되지 않음...
print(error_date)
len(error_date)

['20210302', '20210401', '20210601', '20210804', '20210901', '20211001', '20211102', '20211201']


8

## *** 중복 지우기

In [16]:
url_table = pd.read_csv('url_dataset_2021.csv')
url_table = url_table.drop_duplicates().reset_index(drop = True) #3557개 -> 3472개로 줄음

In [9]:
url_table

,title,url,press,date
0,"올해는 ESG 정보공시 표준화 원년, 압박 더 거세진다",https://news.naver.com/main/read.naver?mode=LS...,머니투데이,2021.01.01.
1,"서학개미 사로잡은 美 ESG ETF, 올해도 高高할까",https://news.naver.com/main/read.naver?mode=LS...,매일경제,2021.01.01.
2,"[신년사]방문규 수은 행장 ""수출 6000억달러 선봉장될 것""",https://news.naver.com/main/read.naver?mode=LS...,뉴시스,2021.01.01.
3,[韓산업 대전환 ①] 격변기의 시대…대전환 앞당겼다,https://news.naver.com/main/read.naver?mode=LS...,아이뉴스24,2021.01.01.
4,"펀드사태 혼쭐난 금융권, 외부 `포청천` 영입",https://news.naver.com/main/read.naver?mode=LS...,매일경제,2021.01.01.
...,...,...,...,...
3466,"국립공원공단, 제1차 '국립공원공단 ESG 위원회' 개최",https://news.naver.com/main/read.naver?mode=LS...,연합뉴스,6일 전
3467,ESG 열풍 이어지는데…‘나쁜 기업’ 투자하는 BAD ETF 시선 집중,https://news.naver.com/main/read.naver?mode=LS...,매경이코노미,6일 전
3468,"가스안전공사, 조직개편…미래성장실·ESG경영처 신설",https://news.naver.com/main/read.naver?mode=LS...,전자신문,7일 전
3469,"동원개발, 새로운 CI 개발…'ESG+S 경영' 선포",https://news.naver.com/main/read.naver?mode=LS...,국민일보,7일 전


In [10]:
# 수집 데이터 저장
url_table.to_csv('url_dataset_2021_drop.csv', encoding = 'utf-8-sig', index=False)

In [11]:
# 수집 데이터 저장
url_dataset.to_csv('url_dataset_2021.csv', encoding = 'utf-8-sig', index=False)

NameError: name 'url_dataset' is not defined

In [12]:
#### 기사 수집에 문제 있는 날짜들
url_collect('esg', '20200907', '20200907')

TypeError: url_collect() missing 1 required positional argument: 'page_num'

# 2) 본문 crawling

In [4]:
url_table = pd.read_csv('url_dataset_2021_drop.csv')
url_table2 = pd.read_csv('url_dataset_2021_drop.csv')

text_url = url_table
text_url['content'] = np.nan

In [5]:
data = list(url_table2.itertuples(index=False, name=None))
data

sql = """
INSERT INTO esg_url_crawl(
    title, url, press, date) VALUES(%s,%s,%s,%s)
"""

cursor.executemany(sql, data)
conn.commit()

In [ ]:
text_url

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}

for index, url in tqdm(enumerate(text_url['url'])):
    print(index, url)
    
    try :
        resp = requests.get(url, headers = headers)
        html = BeautifulSoup(resp.text, "html.parser")
        content = html.find_all('div', {'class' : '_article_body_contents article_body_contents'})[0].text
        print(content)
        text_url.iloc[index, 2] = content
        sql = """INSERT INTO esg_url_crawl(content) VALUES(%s)"""
        cursor.executemany(sql, data)
        conn.commit()
    except :
        text_url.iloc[index, 2] = "ERROR"

0it [00:00, ?it/s]

0 https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=008&aid=0004522316
엔터 치세요 :





	
	[머니투데이 대담=박재범 증권부장, 정리=황국상 기자  ] [[포스트 코로나, ESG가 경제 시스템을 바꾼다]산업·금융·법조계 ESG 전문가 대담]/사진제공=이미지투데이2020년은 ESG(환경·사회적 가치·지배구조) 논의의 원년으로 평가된다. 포털 사이트 다음(Daum)에서 ‘ESG’를 키워드로 해서 검색되는 뉴스 건수만 2만9200건(2020년 12월24일 기준)에 이른다. 2000년 1월부터 2019년 12월말까지, 20년간 나온 전체 ESG 뉴스 건수(약 1만2600건)의 2배를 넘는다. ESG가 주목을 받은 것은 지난해 전세계를 뒤흔든 코로나19(COVID-19) 때문이었다. 전염병이 경제 시스템 전반에 충격을 가하자 정부와 기업들은 그간 눈여겨 보지 않았던 비재무적 리스크 요인에 대한 관리 필요성을 절감했다. 주요 기업이나 금융사들이 ESG를 전사 리스크 관리나 전략기획 차원에서 다루기 시작한 것도 2020년부터다. 올해는 어떨까. 머니투데이는 2021년 신축년을 맞아 기업과 금융, 법조계 등 다양한 분야에서 활동하는 ESG 전문가들을 초빙해 ‘ESG 규제환경의 변화 - 기업·금융의 갈 길’이라는 주제로 대담을 진행했다.이번 대담에는 권기준 SK 수펙스협의회 SV위원회 부사장, 김성우 김앤장법률사무소 환경에너지연구소장, 우상현 KB은행 CIB(기업투자금융) 전무 겸 KB증권 IB(투자은행) 부사장, 윤석모 삼성증권 리서치센터장 겸 ESG연구소장이 참여했다.(이상 가나다 순) 코로나19가 확산되고 있는 시기임을 감안해 이번 대담은 박재범 머니투데이 증권부장 진행에 따라 온라인 화상회의로 진행됐다.━돈 버는 방식의 변화… '해 안끼치고 돈 벌기'→'착하게 돈 벌기'━ESG는 경제 중심, 기업 활동 중심에 들어와 있다. 이에 비해 아직 일반 국민의 체감도는 떨어진다. 

엔터 치세요 :





작년 `착한 투자`에 30조 몰려인베스코솔라 年 220% 급등美기업 IPO 열기도 뜨거워`로빈후드`등 줄줄이 상장대기 올해도 뉴욕증시에서는 ESG 상장지수펀드(ETF)와 기업공개(IPO)주가 서학개미(해외 증시에 투자하는 개인투자자)의 눈길을 끌 것으로 보인다. ESG는 회계 장부상 드러나는 재무제표 외에 기업 비재무 요소인 환경·사회·지배구조 가치를 줄인 말이다. 새롭게 증시에 문을 연 기업 중에서도 쟁쟁한 업체가 많다.우선 ESG 관련 인베스코 솔라 ETF는 지난달 31일(현지시간) 뉴욕증시 마감 가격 기준으로 작년 한 해 시세가 220% 올랐다. 해당 ETF는 엔페이즈에너지와 솔라에지, 퍼스트솔라, 징코솔라 등 태양에너지 기업들 주가를 추종한다. '파리기후변화협약 복귀'를 선언한 미국 조 바이든 정부가 이달 출범하면 친환경 에너지 부문이 상승세를 탈 것이라는 투자자들의 기대를 받는다. 테슬라와 중국 전기차 '니오', 전기차 배터리용 리튬 공급업체 '앨버말' 등을 담고 있는 퍼스트트러스트 나스닥 클린에지 ETF도 177% 올랐다. 글로벌 증시 데이터분석업체 팩트셋이 추산한 바에 따르면 지난해 뉴욕 증시에서는 투자자들이 ESG 관련 ETF에 최소 274억달러(약 30조원)를 쏟아부었다. 이는 사상 최대 규모로 직전 해인 2019년의 두 배 수준이다.   뉴욕증시 ESG ETF는 100여 개인데 지난해 1월부터 12월 16일까지 새로 출시된 것만 31개로, 2019년 전체(16개)의 2배에 달한다. 대표적인 사례가 지난해 7월 나온 '스탠더드앤드푸어스(S&P) 500지수 ESG ETF'다. 출시 넉 달 만에 9000만달러를 끌어모았다.올해도 기업 상장 열기가 이어질 것으로 보인다. 제프 자코프스키 JP모건 미국 증시 수석전략가는 "지금처럼 금리가 거의 0%인 상황에서는 물가상승률 이상 수익을 낼 수 있는 자산이 많지 않은데 그나마 있다면 미국 주식, 특히 IPO 상장 주식이 대표적"이라면서 매수세가 몰릴 수밖에 없다고 분석했다. 이런 

엔터 치세요 :





이재용 삼성전자 부회장은 지난달 30일 서울고법 형사1부 심리로 진행된 국정농단 파기환송심 결심공판에서 최후진술을 통해 "준법을 넘어 최고 수준 투명성과 도덕성을 갖춘 회사로 만들겠다"고 말했다. /남용희 기자2021년에도 경영 위기 계속…삼성은 사법 리스크 고민[더팩트ㅣ이성락 기자] 2021년 신축년 새해가 밝았지만, 재계 총수들의 어깨는 여전히 무겁다.올 한 해 글로벌 기업 간 신사업 경쟁이 그 어느 때보다 치열해질 것으로 예상되면서 신성장동력을 확보하는 데 매진해야 하는 상황이지만, 신종 코로나바이러스 감염증(코로나19) 팬데믹 등 대내외 불확실성이 올해까지 지속될 가능성이 높아지면서 고심이 깊어지는 분위기다.기업별로 풀어야 할 과제도 적지 않다. 삼성의 경우 수년째 발목을 잡았던 '사법 리스크'가 여전히 부담 요소로 남아있다.1일 법조계와 재계에 따르면 오는 18일 이재용 삼성전자 부회장의 파기환송심 선고공판이 열린다. 이 부회장의 향후 거취를 결정짓는 이날 판결을 앞두고 삼성은 초긴장 상태다.이 부회장은 지난해 '사법 리스크' 속에 쉼 없는 현장 경영으로 내실 다지기에 총력을 기울였다. 그는 코로나19 사태 이후 주요 사업장을 차례대로 방문하며 임직원을 격려하는 것은 물론, 중국·네덜란드·베트남 등 재계 총수 가운데 사실상 유일하게 글로벌 현장 경영에 나서기도 했다.'총수 리더십'을 바탕으로 지난해 안정적인 사업포트폴리오 구축과 비대면 수요 확대 등으로 매분기 호실적을 달성한 삼성전자는 코로나19 확산과 미중 무역갈등 지속 등 경영 불확실성에도 위기 대응에 성공했다는 평가를 받는다.삼성은 물론 경제계 안팎에서는 자칫 지난 2017년 '초유의 총수 부재 사태'가 재연될 경우 삼성의 미래 경쟁력이 떨어질 수밖에 없다는 우려의 목소리가 나온다. 실제로 글로벌 기업들이 앞다퉈 대규모 신규투자에 열을 올리고 있는 것과 달리 삼성은 지난 2017년 4월 7일 치러진 1심 첫 공판 이후 지금까지 국정농단 재판에 발목을 잡힌 4년여 동안 M

엔터 치세요 :





'ESG 전도사' 최태원 이어 이재용·정의선·구광모·신동빈·최정우도 가세지난 2019년 말 중국 우한에서 발생한 코로나19는 '글로벌 퍼펙트스톰'을 일으켰다. 이에 따라 글로벌 가치사슬(Global Value Chain)의 개념도 바뀌고 있다. 코로나19가 언택트 시대를 앞당기고 디지털 전환의 촉매제로 작용하면서다. 미국과 중국을 대척점으로 하는 신냉전도 글로벌 경제 생태계에 엄청난 후폭풍을 낳고 있다. 수출의존도가 높은 한국 경제에 엄청난 변수다. 반(反)기업법 쓰나미 역시 경영 환경에 상당한 영향을 미치고 있다. 이 같은 요인은 2021년 신축년(辛丑年)에 더 심화될 가능성이 높다. 이제 한국 경제는 전환 시대를 넘어 지형도를 바꾸는 변혁기에 들어설 전망이다. [편집자 주][아이뉴스24 강길홍 기자] "인류는 지금 글로벌 환경·사회적 위기에 팬데믹까지 더해진 커다란 도전에 직면해 있다. 이 도전들은 글로벌 사회의 포괄적이고도 조화로운 대응을 요구하고 있다. 무엇보다 기업들이 친환경 사업, 사회적 가치, 신뢰받는 지배구조 등을 추구하는 ESG 경영으로 근본적인 변화를 이뤄 나가야한다."'ESG 전도사'로 불리는 최태원 SK그룹 회장이 지난달 열린 '상하이 포럼 2020'에서 개막 연설을 통해 한 말이다. 최 회장이 강조한 것처럼 ESG 경영은 이제 선택이 아닌 생존을 위한 필수 조건이 되고 있다.ESG는 비재무적 요소인 환경(Environment), 사회(Social), 지배구조(Governance)에서 얼마나 많은 가치를 창출하고 있는지를 뜻한다. 착한 투자를 표방하는 펀드들은 ESG를 소홀히 하는 기업들의 주식을 사들이지 않고, 글로벌 기업들 가운데 ESG 기준에 미달하는 업체들과 거래를 하지 않는 경우도 있다.이에 국내 기업들도 앞 다퉈 ESG 경영 방침을 발표하고, 전담조직을 신설하는 등 본격적으로 ESG 경영을 추구하고 있다. 가장 눈에 띄는 곳은 역시 SK그룹이다. 최태원 SK그룹 회장은 'ESG 전도사'로 불릴 만큼 오래

엔터 치세요 :





2일 정유업계에 따르면 지난해 국내 정유4사는 코로나19 여파에 직격탄을 맞으며 총 5조 원에 육박하는 적자를 기록하는 등 '최악의 해'를 보냈다. /더팩트 DB친환경 사업 진출·신사업 확대 등 수익 활로 찾기 나서[더팩트ㅣ이한림 기자] 지난해 사상 최악의 실적으로 한 해 농사를 망친 국내 정유4사(SK이노베이션·GS칼텍스·현대오일뱅크·에쓰오일)가 올해 반등을 노리고 있다. 수익성 높은 사업에 대한 비중을 높이고 유가에 영향을 크게 받지 않는 신사업군을 확대해 활로를 찾겠다는 방침이다.2일 정유업계에 따르면 국내 정유4사는 지난해 3분기까지 4조8074억 원의 누적 적자를 기록했다. 4분기에도 코로나19 재확산 여파가 지속되면서 적자 기조는 탈피하지 못할 전망이다. 최근 3년 간 수조원대 영업이익을 내며 호황기를 누렸던 것과 대조적인 결과이다. 더욱이 코로나 펜데믹 여파에 내년에도 업황 개선은 기대하기 어려울 것으로 보인다.업계에서는 정유4사가 5조 원대에 육박하는 누적 적자를 기록한 것에 대해 국제 유가가 요동치고 석유 제품 수요가 급감해 재고 관련 손실이 불어난 영향으로 보고 있다.특히 정유사 매출의 상당부분을 차지하는 항공유나 선박연료로 쓰이는 벙커C유 등 석유 제품 수요의 감소는 사상 초유의 '마이너스(-)' 정제마진으로 이어지면서 최악의 해를 보내게 됐다. 최종 석유제품 가격에서 원유나 수송·운영비 등을 뺀 정제마진은 일반적으로 4달러를 손익분기점으로 보고 있는 것을 감안하면 공장을 돌리면 돌릴수록, 석유제품을 팔면 팔수록 손해를 보는 구조로 이어졌다.하반기 다소 업황이 개선되면서 적자 폭을 줄였으나 내년에도 석유 제품 수요 증가를 기대하기 어렵다는 전망이 나오면서 실적 부진을 완전히 털어내기는 어렵다는 시각이 지배적이다.이에 정유사들은 위기를 기회로 삼겠다는 입장이다. 강력한 체질 개선 천명을 통해 기존 사업 구조를 변경하고, 기후 변화 이슈 증가로 수요가 확대되는 친환경 제품 생산에 주력하거나 전기차, 주유소 인프라 사업 

엔터 치세요 :





최태원 SK그룹 회장주요 기업 총수들이 신년사를 통해 최근 재계 화두인 ‘ESG(환경·사회·지배구조) 경영’으로 코로나19 위기를 돌파하자고 입을 모았다.3일 업계에 따르면 최태원 SK그룹 회장은 구성원들에게 메일로 보낸 신년사에서 “여러 부침 속에서도 SK가 지금의 모습으로 성장할 수 있었던 것은 우리만 잘해서가 아니라 사회가 허락한 기회와 응원 덕”이라며 “사회와 공감하며 문제 해결을 위해 함께 노력하는 ‘새로운 기업가 정신’이 필요할 때”라고 말했다. 그는 “우리는 사회가 원하는 것이 무엇인지 제대로 헤아리지 못했다. 기업의 역할을 다시 생각하게 된다”면서 “기후 변화나 팬데믹(대유행) 같은 대재난은 사회의 가장 약한 곳을 먼저 무너뜨린다. 기업도 더이상 이러한 문제로부터 자유로울 수 없다”고 강조했다.조현준 효성 회장조현준 효성그룹 회장은 “효성을 고객이 믿고 인정하는 ‘프리미엄 브랜드’로 만들어야 한다”고 강조했다. 그는 “지난해 ESG 평가에서 효성 5개사가 A등급 이상을 받는 성과를 올렸다”면서 “한층 더 노력해 환경보호와 정도경영, 투명경영을 선도하고 협력사들과의 동반성장을 추구함으로써 국민으로부터 사랑과 신뢰를 받는 효성이 되도록 하자”고 말했다.정지선 현대백화점그룹 회장정지선 현대백화점그룹 회장은 “고객이 궁극적으로 원하는 가치를 찾아 사업 프로세스와 일하는 방식을 바꿔 미래 성장을 준비하자”고 제안했다. 그는 “코로나19와 수년째 지속되는 경기침체 그리고 디지털 전환을 축으로 한 산업 패러다임의 급변으로 어려운 사업 환경이 예상된다. 변화의 흐름을 읽고 잠재적인 고객의 니즈를 찾아내는 역량이 무엇보다 중요해지고 있다”고 강조했다.유경선 유진그룹 회장유경선 유진그룹 회장도 “팬데믹을 넘기 위해서는 ‘팬덤’(열성 팬층)이 필요하다”면서 “마치 방탄소년단(BTS)이 아미(Army)라는 팬클럽과 함께 세계적인 팬덤을 이끄는 것처럼 현재의 역량과 시선을 한 단계 올린다면 주주와 고객, 임직원과 사회구성원 모두가 환호하는 탁월

엔터 치세요 :





툭하면 경영권 공격거래업체 주문 끊기도미국 최대 에너지기업 엑슨모빌은 지난달 7일 행동주의 투자기업 엔진넘버원으로부터 ESG(환경·사회·지배구조) 활동을 강화하지 않으면 이사진 4명을 갈아치우겠다는 경고를 받았다. 엔진넘버원은 엑슨모빌에 온실가스 감축 방안을 내놓고 신재생에너지 사업을 늘리라고 요구했다. 압박이 커지자 엑슨모빌은 1주일 만에 평상시 가스전에서 배출되는 잉여가스를 태우는 플레어링을 2030년부터 아예 끊기로 하는 등 온실가스 배출 강도를 줄이겠다고 약속했다.글로벌 기업 중에도 ESG 리스크로 고민하는 곳이 늘고 있다. ESG 경영에 소홀하다는 이유로 소비자와 거래처는 물론 투자기업들까지 등을 돌리고, 엑슨모빌처럼 경영권 공격을 받는 사례가 발생하고 있어서다.구글은 지난달 말 사우디아라비아 국영 석유기업 아람코와 제휴해 클라우드컴퓨팅 시장에 진출하는 과정에서 반발을 겪었다. 구글의 데이터 서비스가 사우디 왕실을 비판하는 이들을 감시하는 데에 악용될 수 있다는 우려가 제기돼서다. 지난해 6월엔 아마존이 미국 경찰과 이미 계약한 안면인식 기술 서비스 사업을 1년간 유예하기로 했다. 미국 경찰의 인종차별 논란이 커지자 아마존이 자사 기술을 인종차별 도구로 팔았다는 비판 여론이 들끓었기 때문이다. 모두 ESG 리스크가 사업 발목을 잡은 사례다.세계 최대 의료용 장갑 제조업체인 말레이시아의 탑글러브코퍼레이션은 올 들어 ‘지속가능한 근로환경’ 문제로 홍역을 치르고 있다. 지난해 7월엔 미국이 노동착취 우려가 있다며 탑글러브의 자회사 두 곳에서 생산한 장갑을 수입제한 목록에 올렸다.지난해 10월부터는 열악한 근무환경 때문에 공장 내 신종 코로나바이러스 감염증(코로나19) 확진자가 폭증해 공장을 폐쇄하고 당국의 대대적 조사를 받았다. 탑글러브가 부랴부랴 1억링깃(약 270억원)을 들여 근로환경을 개선하겠다고 발표했지만 이젠 거래기업들이 발을 빼는 분위기다. 지난달 27일엔 크리넥스, 하기스 등 생활용품 브랜드를 운영하는 주요 거래처 킴벌

In [19]:

text_url.iloc[0,-1]

"\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n\t\n\t[머니투데이 대담=박재범 증권부장, 정리=황국상 기자  ] [[포스트 코로나, ESG가 경제 시스템을 바꾼다]산업·금융·법조계 ESG 전문가 대담]/사진제공=이미지투데이2020년은 ESG(환경·사회적 가치·지배구조) 논의의 원년으로 평가된다. 포털 사이트 다음(Daum)에서 ‘ESG’를 키워드로 해서 검색되는 뉴스 건수만 2만9200건(2020년 12월24일 기준)에 이른다. 2000년 1월부터 2019년 12월말까지, 20년간 나온 전체 ESG 뉴스 건수(약 1만2600건)의 2배를 넘는다. ESG가 주목을 받은 것은 지난해 전세계를 뒤흔든 코로나19(COVID-19) 때문이었다. 전염병이 경제 시스템 전반에 충격을 가하자 정부와 기업들은 그간 눈여겨 보지 않았던 비재무적 리스크 요인에 대한 관리 필요성을 절감했다. 주요 기업이나 금융사들이 ESG를 전사 리스크 관리나 전략기획 차원에서 다루기 시작한 것도 2020년부터다. 올해는 어떨까. 머니투데이는 2021년 신축년을 맞아 기업과 금융, 법조계 등 다양한 분야에서 활동하는 ESG 전문가들을 초빙해 ‘ESG 규제환경의 변화 - 기업·금융의 갈 길’이라는 주제로 대담을 진행했다.이번 대담에는 권기준 SK 수펙스협의회 SV위원회 부사장, 김성우 김앤장법률사무소 환경에너지연구소장, 우상현 KB은행 CIB(기업투자금융) 전무 겸 KB증권 IB(투자은행) 부사장, 윤석모 삼성증권 리서치센터장 겸 ESG연구소장이 참여했다.(이상 가나다 순) 코로나19가 확산되고 있는 시기임을 감안해 이번 대담은 박재범 머니투데이 증권부장 진행에 따라 온라인 화상회의로 진행됐다.━돈 버는 방식의 변화… '해 안끼치고 돈 벌기'→'착하게 돈 벌기'━ESG는 경제 중심, 기업 활동 중심에 들어와 있다. 이에 비해 아직 일반 국민의 체감도는 떨어진다. ‘뜬구름 잡는’ 선언처럼 느끼는 이도 적잖

In [21]:
# 수집되지 않은 기사 수 25개
len(text_url[text_url['content'] =='ERROR'])

25

In [23]:
text_url.to_csv('text_url_2021.csv', encoding = 'utf-8-sig', index = False)

In [39]:
html.find_all('div', {'class' : '_article_body_contents article_body_contents'})[0].text

'\n\n\n\n\n\t\n\tKB증권 \'환경·사회·지배구조 이슈분석\' 보고서KB증권[연합뉴스TV 제공]    (서울=연합뉴스) 임미나 기자 = KB증권은 30일 세계적인 자산운용사 SSGA(State Street Global Advisors)가 최근 주요 글로벌 기업들에 ESG(환경·사회·지배구조) 문제 개선을 공식 요구하고 나섰다며 향후 국내 기업에도 영향을 줄 것으로 전망했다.    김준섭 연구원은 \'ESG 이슈분석\' 보고서에서 "세계 3대 운용사인 SSGA가 지난 28일 ESG 기준에 미달한 대기업을 대상으로 ESG 관련 문제를 개선할 것을 요구하는 내용의 서신을 발송했다"며 "SSGA는 (EGS 관련) 주주관여 활동을 지속하는 한편 올해부터 의결권을 적극적으로 행사하겠다고 밝혔다"고 전했다.    김 연구원은 SSGA가 이 서신에서 지난해 도입한 ESG 평가 기준(R-Factor)을 공개하면서 2022년까지 모든 기업을 대상으로 ESG 관여 전략을 시행한다는 계획(로드맵)을 밝혔다고 덧붙였다.    SSGA는 2017년 이후 다양성 및 성 평등 문제를 중심으로 ESG 활동을 추진해 왔으며, 그 결과 583개 기업이 이사회에 여성 이사를 선임한 것으로 알려져 있다.    김 연구원은 "SSGA는 평가 기준을 통해 6천여개 기업과 산업을 대상으로 표준화된 ESG 평가를 시도했다는 점에서 의미가 있다"며 향후 ESG 평가 표준이 형성될 것으로 기대했다.    그는 "현재 여러 ESG 지표의 난립과 수많은 규약이 ESG 투자의 걸림돌로 작용하고 있으나, 평가 방식이 표준화되면서 ESG 투자가 활성화할 전망"이라고 분석했다.    또 SSGA가 이번에 서신을 보낸 대상 가운데 한국 기업은 없어 올해 국내에서 이와 관련한 주총 의결권 행사 이슈는 없을 것으로 예상했지만, 조만간 국내 기업에도 SSGA의 전략이 영향을 줄 것으로 내다봤다.    그는 SSGA가 국내 운용사와 협업해 상장지수펀드(ETF)를 출시하고 전주에 사무소를 여는 등 국내 노출도를 높이

In [23]:
#db 입력
conn = pymysql.connect(host='127.0.0.1', user='root', password='1234',db='esg_db', charset='utf8mb4')
cursor = conn.cursor()

text_url = pd.read_csv('text_url_2021.csv')

content = text_url['content']
content
data = list(content.itertuples(index=False, name=None))
#data

sql = """
INSERT INTO esg_url_crawl(
    content) VALUES(%s)
"""

cursor.executemany(sql, content)
conn.commit()

AttributeError: 'Series' object has no attribute 'itertuples'

0       \n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...
1       \n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...
2       \n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...
3       \n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...
4       \n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...
                              ...                        
3467    \n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...
3468    \n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...
3469    \n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...
3470    \n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...
3471    \n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...
Name: content, Length: 3472, dtype: object